# Transfer Learning Model Training with MobileNet for Casting Defect Detection use case 

## Dataset available here: https://www.kaggle.com/ravirajsinh45/real-life-industrial-dataset-of-casting-product

### Importing the required libraries

In [ ]:
import tensorflow.keras
import tensorflow as tf
import numpy as np
from  tensorflow.keras.utils import multi_gpu_model 
from tensorflow.keras import models
from tensorflow.keras.preprocessing import image
from tensorflow.keras import applications,layers,models,optimizers

import warnings
warnings.filterwarnings('always')

In [ ]:
import os
os.listdir("../Data")

### Unzipping the files

In [ ]:
import zipfile
with zipfile.ZipFile("../Data/casting_data.zip", 'r') as zip_ref:
    zip_ref.extractall("../Data/castingData")

In [ ]:
mainpath='../Data/castingData/'

In [ ]:
path=mainpath+'train/'
path2=mainpath+'validation/'
img_height, img_width=224,224
batch_size=15

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Training Data Generator

In [ ]:
train_datagen = ImageDataGenerator(rescale=1. / 255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(path,target_size=(img_height, img_width),
batch_size=batch_size,class_mode='categorical')

### Validation Data Generator

In [ ]:
val_datagen = ImageDataGenerator(rescale=1. / 255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

val_generator = val_datagen.flow_from_directory(path2,target_size=(img_height, img_width),
batch_size=batch_size,class_mode='categorical')

### Transfer Learning

In [ ]:
model=applications.MobileNet(weights = "imagenet", include_top=False,input_shape = (img_width, img_height, 3))

In [ ]:
x = model.output
x = layers.Flatten()(x)
x = layers.Dense(1024, activation="relu")(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(2, activation="softmax")(x)

# creating the final model 
model_final = models.Model(model.input, predictions)

In [ ]:
for j in model_final.layers[:-5]:
    j.trainable=False

In [ ]:
model_final.summary()

### Model Compilation

In [ ]:
# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adam(lr=0.0002), metrics=["accuracy"])

### Model Training

In [ ]:
model_final.fit_generator(train_generator,validation_data=val_generator,
                          steps_per_epoch=20,validation_steps=5,epochs=20)

### Model conversion to ONNX

In [ ]:
# convert to onnx model
import keras2onnx
onnx_model = keras2onnx.convert_keras(model_final, model.name)
temp_model_file = '../Model/castingDefectModelViaJNB.onnx'
keras2onnx.save_model(onnx_model, temp_model_file)